# UK Biobank lipids phenotypes and covariates

In this notebook we review the available UK Biobank data for lipids and prepare our GWAS phenotypes and covariates.
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30690
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30760
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30780
  * https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=30870

# Setup

<div class="alert alert-block alert-warning">
This notebook will run correctly on the UK Biobank Research Analysis Platform.
</div>

In [ ]:
lapply(c('lubridate', 'skimr', 'tidyverse'),
       function(pkg) { if(! pkg %in% installed.packages()) { install.packages(pkg)} } )

In [ ]:
library(lubridate)
library(skimr)
library(tidyverse)

In [ ]:
## Plot setup.
theme_set(theme_bw(base_size = 16)) # Default theme for plots.

#' Returns a data frame with a y position and a label, for use annotating ggplot boxplots.
#'
#' @param d A data frame.
#' @return A data frame with column y as max and column label as length.
get_boxplot_fun_data <- function(df) {
  return(data.frame(y = max(df), label = stringr::str_c('N = ', length(df))))
}

## Retrieve and load the data extract

<div class="alert alert-block alert-warning">
This section assumes the availability of CSVs created via notebook <kbd>ukb_lipids_phenotypes_retrieval.ipynb</kbd> and uploaded to the UKB RAP project.
</div>

In [ ]:
# The CSV is well-formed and visible.
system('cat /mnt/project/drug_mapping.csv', intern = TRUE)

In [ ]:
# But this R command does not work.
read_csv('/mnt/project/drug_mapping.csv')

From the shell, run:
```
dx download drug_mapping.csv
dx download lipids.csv
dx download w7089_20210809.csv
dx download bridge_7089_31063.tsv
```

Note: we cannot do this via the `system` command because it requires interaction to choose the version when multiple versions are present.

In [ ]:
statin_drugs <- read_csv('drug_mapping.csv')

In [ ]:
withdrawn_eids <- read_tsv('w7089_20210809.csv', col_names = c('eid'))

In [ ]:
eid_to_sampleid <- read_tsv('bridge_7089_31063.tsv')

In [ ]:
raw_pheno <- read_csv('lipids.csv')

In [ ]:
spec(raw_pheno)

In [ ]:
#skim(raw_pheno)

At this time, we are only interested in the first instance.

In [ ]:
pheno <- raw_pheno %>% select(eid, p31_Sex, contains('_i0_'))

In [ ]:
skim(pheno)

## Omit participants who have withdrawn from the study

In [ ]:
nrow(pheno)

In [ ]:
pheno <- pheno %>%
    filter(!eid %in% withdrawn_eids$eid)

In [ ]:
nrow(pheno)

# Examine the lipids data 

In [ ]:
table(pheno$p30692_i0_Cholesterol_aliquot, useNA = 'always')

In [ ]:
pheno %>%
    mutate(
        missing = is.na(p30690_i0_Cholesterol_mmol_L)
    ) %>%
    group_by(pheno$p30693_i0_Cholesterol_correction_level, missing) %>%
    summarize(count = n())

In [ ]:
pheno %>%
    mutate(
        missing = is.na(p30690_i0_Cholesterol_mmol_L)
    ) %>%
    group_by(p30694_i0_Cholesterol_correction_reason, missing) %>%
    summarize(count = n())

In [ ]:
pheno %>%
    mutate(
        missing = is.na(p30690_i0_Cholesterol_mmol_L)
    ) %>%
    group_by(p30695_i0_Cholesterol_missing_reason, missing) %>%
    summarize(count = n())

## Plot the lipids data 

In [ ]:
options(repr.plot.height = 12, repr.plot.width = 16)

pheno %>%
    filter(!is.na(p21003_i0_Age_when_attended_assessment_centre_years)) %>%
    ggplot(aes(x = cut_width(p21003_i0_Age_when_attended_assessment_centre_years, width = 10, boundary = 0), y = p30690_i0_Cholesterol_mmol_L)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    coord_flip() +
    xlab('age') +
    labs(title = str_glue('Instance 0 measurement per person, by age'),
         caption = 'Source: UK Biobank data')

In [ ]:
pheno_check_long <- pheno %>%
    select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('mmol_L')) %>%
    pivot_longer(
        cols = ends_with('mmol_L'),
        names_to = c('instance', 'measurement'),
        names_pattern = 'p\\d+_(i\\d+)_(.*)_mmol_L',
        values_to = 'mmol_L') %>%
    # Convert to units used by AoU measurements.
    # Formula is from https://www.ncbi.nlm.nih.gov/books/NBK83505/
    mutate(
        mg_dl = case_when(
            measurement == 'HDL_cholesterol' ~ mmol_L * 38.67,
            measurement == 'Cholesterol' ~ mmol_L * 38.67,
            measurement == 'LDL_direct' ~ mmol_L * 38.67,
            measurement == 'Triglycerides' ~ mmol_L * 88.57,
            TRUE ~ NA_real_
        )
    ) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('correction_reason')) %>%
      pivot_longer(
          cols = ends_with('correction_reason'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_correction_reason',
          values_to = 'correction_reason')) %>% 
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('missing_reason')) %>%
      pivot_longer(
          cols = ends_with('missing_reason'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_missing_reason',
          values_to = 'missing_reason')) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('correction_level')) %>%
      pivot_longer(
          cols = ends_with('correction_level'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_correction_level',
          values_to = 'correction_level')) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('aliquot')) %>%
      pivot_longer(
          cols = ends_with('aliquot'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_aliquot',
          values_to = 'aliquot')) %>%
    inner_join(
      pheno %>%
      select(eid, p21003_i0_Age_when_attended_assessment_centre_years, ends_with('reportability')) %>%
      pivot_longer(
          cols = ends_with('reportability'),
          names_to = c('instance', 'measurement'),
          names_pattern = 'p\\d+_(i\\d+)_(.*)_reportability',
          values_to = 'reportability'))

In [ ]:
# Check the result of the join.
(dim(pheno_check_long))
(nrow(pheno) * 4)
stopifnot(nrow(pheno_check_long) == nrow(pheno) * 4)

In [ ]:
# Uncomment the line below to see row level data.
#head(pheno_check_long)

In [ ]:
pheno_check_long %>%
    filter(!is.na(mg_dl)) %>%
    group_by(missing_reason) %>%
    summarize(count = n())

In [ ]:
pheno_check_long %>%
    group_by(measurement) %>%
    summarize(
        count = n(),
        missing = sum(is.na(mg_dl)),
        median = median(mg_dl, na.rm = TRUE),
        mean = mean(mg_dl, na.rm = TRUE),
        stddev = sd(mg_dl, na.rm = TRUE)
    )

In [ ]:
options(repr.plot.height = 18, repr.plot.width = 16)

pheno_check_long %>%
    filter(!is.na(p21003_i0_Age_when_attended_assessment_centre_years)) %>%
    ggplot(aes(x = cut_width(p21003_i0_Age_when_attended_assessment_centre_years, width = 15, boundary = 0), y = mg_dl)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
#    scale_y_log10(breaks = scales::pretty_breaks(n = 10)) +  # Uncomment if the data looks skewed.
    coord_flip() +
    facet_wrap(~ measurement, nrow = length(unique(pheno_check_long$measurement)), scales = 'free_x') +
    xlab('age') +
    labs(title = str_glue('Instance 0 measurement per person, by age'),
         caption = 'Source: UK Biobank data')

# Determine statin use 

In [ ]:
pheno %>%
    select(p20003_i0_Treatment_medication_code) %>%
    filter(str_detect(p20003_i0_Treatment_medication_code, 'statin')) %>%
    head()


In [ ]:
statin_drugs

In [ ]:
(statin_regex = str_c(statin_drugs$drug_name, collapse = '|'))

In [ ]:
str_detect(pheno$p20003_i0_Treatment_medication_code[100], statin_regex)

In [ ]:
pheno <- pheno %>%
    mutate(
        statin_use = case_when(
            str_detect(p20003_i0_Treatment_medication_code, statin_regex) ~ 1,
            TRUE ~ 0
        )
    )

table(pheno$statin_use, useNA = 'always')

# Age covariate

In [ ]:
table(pheno$p21003_i0_Age_when_attended_assessment_centre_years, useNA='always')

In [ ]:
pheno <- pheno %>%
    filter(!is.na(p21003_i0_Age_when_attended_assessment_centre_years)) %>%
    mutate(
        age = p21003_i0_Age_when_attended_assessment_centre_years,
        age2 = age ^2
    )

In [ ]:
table(pheno$age, useNA='always')

In [ ]:
table(pheno$age2, useNA='always')

# Sex covariate

In [ ]:
table(pheno$p31_Sex, useNA='always')

In [ ]:
pheno <- pheno %>%
    filter(!is.na(p31_Sex))

In [ ]:
table(pheno$p31_Sex, useNA='always')

## Lipid phenotypes

In [ ]:
nrow(pheno)

Retain only participants with values for all four lipids.

In [ ]:
pheno <- pheno %>%
    filter(!is.na(p30690_i0_Cholesterol_mmol_L)) %>%
    filter(!is.na(p30780_i0_LDL_direct_mmol_L)) %>%
    filter(!is.na(p30760_i0_HDL_cholesterol_mmol_L)) %>%
    filter(!is.na(p30870_i0_Triglycerides_mmol_L))

In [ ]:
nrow(pheno)

Convert from mmol/L to mg/dL. Formula is from https://www.ncbi.nlm.nih.gov/books/NBK83505/

In [ ]:
pheno <- pheno %>%
    mutate(
        Total_cholesterol_mg_dl = p30690_i0_Cholesterol_mmol_L * 38.67,
        LDL_mg_dl = p30780_i0_LDL_direct_mmol_L * 38.67,
        HDL_cholesterol_mg_dl = p30760_i0_HDL_cholesterol_mmol_L * 38.67,
        Triglycerides_mg_dl = p30870_i0_Triglycerides_mmol_L * 88.57
    )

1.	LDL adjustment based on TG/LDL values 
  1.	`If TG > 400, then LDL = NA`
  2.	`If LDL < 10, then LDL = NA`
2.	LDL and TC adjustment based on Statin (Lipid lowering medication)
  1.	`If STATIN is used, LDL_ADJ = LDL/0.7`
  2.	`If STATIN is used, TOTAL_ADJ = TC/0.8`
3.	TG adjustment
  1.	`TG_LOG = log(TG)`

In [ ]:
pheno <- pheno %>%
    mutate(
        LDL_adj_mg_dl = case_when(
            Triglycerides_mg_dl > 400 ~ NA_real_,
            LDL_mg_dl < 10 ~ NA_real_,
            statin_use == 1 ~ LDL_mg_dl / 0.7,
            TRUE ~ LDL_mg_dl
        ),
        Total_cholesterol_adj_mg_dl = ifelse(statin_use == 1, Total_cholesterol_mg_dl / 0.8, Total_cholesterol_mg_dl),
        Triglycerides_log_mg_dl = log(Triglycerides_mg_dl)
    )

In [ ]:
pheno_long <- pheno %>%
    select(eid, age, age2, p31_Sex, ends_with('mg_dl')) %>%
    pivot_longer(
        cols = ends_with('mg_dl'),
        names_to = c('measurement'),
        names_pattern = '(.*)_mg_dl',
        values_to = 'mg_dl')

In [ ]:
pheno_long %>%
    group_by(measurement) %>%
    summarize(
        count = n(),
        missing = sum(is.na(mg_dl)),
        median = median(mg_dl, na.rm = TRUE),
        mean = mean(mg_dl, na.rm = TRUE),
        stddev = sd(mg_dl, na.rm = TRUE)
    )

In [ ]:
options(repr.plot.height = 24, repr.plot.width = 16)

pheno_long %>%
    ggplot(aes(x = cut_width(age, width = 15, boundary = 0), y = mg_dl)) +
    geom_boxplot() +
    stat_summary(fun.data = get_boxplot_fun_data, geom = 'text', size = 4,
                 position = position_dodge(width = 0.9), vjust = -0.8) +
    scale_y_continuous(breaks = scales::pretty_breaks(n = 10)) +
#    scale_y_log10(breaks = scales::pretty_breaks(n = 10)) +  # Uncomment if the data looks skewed.
    coord_flip() +
    facet_wrap(~ measurement, nrow = length(unique(pheno_long$measurement)), scales = 'free_x') +
    xlab('age') +
    labs(title = str_glue('Instance 0 measurement per person, by age'),
         caption = 'Source: UK Biobank data')

# Retain only participants with data in the 200k WES release

In [ ]:
nrow(pheno)

In [ ]:
pheno <- inner_join(pheno, eid_to_sampleid, by = c('eid' = 'eid_7089'))

# TODO add PCs

In [ ]:
skim(eid_to_sampleid)

In [ ]:
nrow(pheno)

## Normalize lipids values

4.	Calculation of residuals – residuals calculated by adjusting for covariates 
  1.	residual calculation Example for LDL: `tmp.ldl$LDL_ADJ.resid <- resid(lm(LDL_ADJ ~ sex+age+age2+PC1+PC2+PC3+PC4+PC5+PC6+PC7+PC8+PC9+PC10+PC11, data = tmp.ldl))`
5.	normalization Example for LDL: `tmp.ldl$LDL_ADJ.norm <- sd(tmp.ldl$LDL_ADJ)*scale(qnorm((rank(tmp.ldl$LDL_ADJ.resid,na.last="keep")-0.5)/length(tmp.ldl$LDL_ADJ.resid)))`

# Provenance 

In [ ]:
devtools::session_info()